# Student Planner:

### This program is design to convert the PDF timetable given to students to a planner in Microsoft Excel, before subsequently exporting to Google Calendar once courses confirmed.

#### From:

![Picture](SSM_Planner/Pic1.png)

#### To Excel:

![Picture](SSM_Planner/Excel9.png)

#### and Google Calendar afterwards.

![Picture](SSM_Planner/Google5.png)

## Mircosoft Excel

1) Run the python program
    - Input start and end page accordingly to your needs.
    - Input the date of first day of school in the correct format as shown.

2) Once completed, look for 'SSM Timetable Planner.xlsx' file. It will be in the same folder from where you had run the program.

3) Open the Excel file and import the VBA code.
    - Developer -> View Code -> File -> Import File
    - Once imported, it will appear in the module folder. Check that the code is present.
    - Developer -> Macros -> Click on the 'Start' macro -> Run
    
4) Once completed, proceed to 'Sheet 2' and add your courses accordingly. Click on the 'Calendar' sheet to visually see your timetable.

5) Once courses have been registered via ISAAC, it's time to export to Google calendar. 'Sheet 2' -> 'Google Format' button

6) Add any description, if necessary. Click 'Export' when ready.

## Google Calendar

1) Open Google Calender

2) Look for 'Other Calendars' on the left side --> '+' button --> Import

3) Look for the CSV file.

4) Select the calendar you which to add.

5) Google Calendar will indicate how many courses were added, depending on how many where added by you.

6) See your newly added timetable.

To use Jupyter to run, follow the steps below.

On the GitHub page --> Right click on the .ipynb file --> Copy link address --> Click [here](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb) --> File --> GitHub --> Paste the link --> Run the code in order

# Install packages and importing

In [ ]:
!pip install tabula-py
!pip install "camelot-py[cv]"
!pip install ghostscript
!pip install excalibur-py
!pip install xlsxwriter
import tabula
import camelot
import pandas as pd
import numpy as np
import openpyxl
import datetime
from datetime import date
from datetime import timedelta
import calendar
import pandas as pd
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo

# Open PDF and read

In [2]:
#ALlow user to choose the file
file = "SSM Timetable_AY21S1_3.pdf"

#Allow user to input page to extract
start = input("Input start page: ")
end = input("Input last page: ")
page = f"{start}-{end}"

#Read the pdf and extract the table
tables = camelot.read_pdf(file, pages=page)
print("\nTotal tables extracted: ", tables.n)

#Remove first row from second page onwards and append
df = tables[0].df
for t in range(1, tables.n):
  data = tables[t].df
  data = data.drop(labels=0, axis=0)
  df = df.append(data)

#Make first row as the header
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

#Changing column name
headerName = ["Course Code", "Title", "Class Type", "Course Type", "Group", "Day", "Time", "Venue", "Remark"]
df.columns = headerName

#Export out first because need to use index_col to make the first row as header
df.to_excel("Table.xlsx")

#Read the excel file
df2 = pd.read_excel("Table.xlsx", index_col=[0])

df2.fillna(method="ffill", inplace=True) #Filling up the empty cells
df2 = df2.dropna(axis=1) #Drop remarks column

for column in df2.columns:
    df2[column] = df2[column].str.replace("\n", "")
    
#Dropping sport psychology for ease
df2 = df2.drop(labels=8, axis=0)

#Splitting the time into start and end time
df2['Start Time'] = df2['Time'].str.split('-', expand=True)[0]
df2['End Time'] = df2['Time'].str.split('-', expand=True)[1]

#Format time to include semicolon
df2['Start Time'] = df2['Start Time'].str[:2] + ":" + df2['Start Time'].str[-2:]
df2['End Time'] = df2['End Time'].str.strip().str[0:2] + ":"+ df2['End Time'].str.strip().str[-2:]

#Delete old time colum
df2 = df2.drop('Time', axis=1)

#Drop any column that is null, i.e. Remarks
df2 = df2.dropna(axis=1)
# df2['Start Time'] = pd.to_datetime(df2['Start Time']).dt.time
# df2['End Time'] = pd.to_datetime(df2['End Time']).dt.time

#Formating the 'Date' column to datetime
week_days= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
date = str(input('\nEnter the first day of school in dd mm yyyy format, with spaces. \n Example: 9 August 2021 = 09 08 2021 \n Date:  '))
day, month, year = date.split(' ') #10 08 2021
startdate = datetime.date(int(year), int(month), int(day)) #2021-08-10

#From startdate, find the next 6 days.
ls = []
all = []
enddate = startdate + timedelta(days=6) #2021-08-16
diff = enddate - startdate

for i in range(diff.days + 1):
    datee = startdate + datetime.timedelta(i)
    k = datee.strftime("%A") #day name
    d = datee.strftime("%d") #day number
    m = datee.strftime("%m") #month number
    y = datee.strftime("%Y") #year number
    ls.append(k)
    all.append(d + m + y) #to get [10082021, 11082021,...] etc

#Put the date in 10/08/2021 format
newls = []
for i in all:
    i = i[0:2] + "/" + i[2:4] + "/" + i[4:]
    newls.append(i)

#Use the new format so can use the .weekday() function
newday = []
for i in newls:
    newlist = list(map(int, i.split('/')))
    #print(newlist)
    day=datetime.date(newlist[2],newlist[1],newlist[0]).weekday() #1
    d = week_days[day][:3].upper()#TUE
    newday.append(d) #['TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN', 'MON']

#put in a new dataframe, then merge
list_of_tuples = list(zip(newls, newday))

df3 = pd.DataFrame(list_of_tuples,
                  columns = ['Date', 'Day'])
df3['Date']= pd.to_datetime(df3['Date'], dayfirst=True)

df2 = pd.merge(df2, df3)

#Export out as excel
df2.to_excel("SSM Timetable Planner.xlsx")
wb = load_workbook(filename = 'SSM Timetable Planner.xlsx')
ws = wb.active

#Delete the first column which shows the row numbers
ws.delete_cols(idx=1,amount=1)
wb.save("SSM Timetable Planner.xlsx")

print("Success. Please check your file explorer.")

Input start page:  11
Input last page:  15



Total tables extracted:  5



Enter the first day of school in dd mm yyyy format, with spaces. 
 Example: 9 August 2021 = 09 08 2021 
 Date:   10 08 2021


Success. Please check your file explorer.
